In [ ]:
from mainnet_launch.pages.autopool_diagnostics.lens_contract import *
from mainnet_launch.data_fetching.get_state_by_block import *
from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *
from mainnet_launch.constants import *

from pprint import pprint
from web3 import Web3


def get_eth_value_by_destination_by_autopool(lens_contract_data: dict):
    dests = []
    autopool_symbol_to_autopool_constant = {a.symbol: a for a in ALL_AUTOPOOLS}

    for autopool_len_contract_data, list_of_destinations in zip(
        lens_contract_data["autopools"], lens_contract_data["destinations"]
    ):
        autopool: AutopoolConstants = autopool_symbol_to_autopool_constant[autopool_len_contract_data["symbol"]]
        for dest in list_of_destinations:
            dest["autopool_base_asset_decimals"] = autopool.base_asset_decimals
            dest["autopool_symbol"] = autopool.symbol
            dests.append(dest)

    return pd.DataFrame(dests)[
        ["vaultAddress", "autoPoolOwnsShares", "actualLPTotalSupply", "autopool_base_asset_decimals", "autopool_symbol"]
    ]


def get_underlying_reserves_by_block(lens_contract_df: pd.DataFrame, block: int, chain: ChainData) -> dict:
    calls = [
        Call(
            destination_vault_address,
            "underlyingReserves()(address[],uint256[])",
            [
                ((destination_vault_address, "underlyingReserves_tokens"), identity_with_bool_success),
                ((destination_vault_address, "underlyingReserves_amounts"), identity_with_bool_success),
            ],
        )
        for destination_vault_address in lens_contract_df["vaultAddress"]
    ]

    acutal_reserves_state = get_state_by_one_block(calls, block, chain)
    return acutal_reserves_state


def fetch_raw_amounts_by_destination_by_autopool(block: int, chain: ChainData) -> pd.DataFrame:
    data = get_full_destination_pools_and_destinations_at_one_block(chain, block)

    lens_contract_df = get_eth_value_by_destination_by_autopool(data)

    acutal_reserves_state = get_underlying_reserves_by_block(lens_contract_df, block, chain)

    # 'debtValueHeldByVault', might want for sanity check
    lens_contract_df["portion_ownership"] = (
        lens_contract_df["autoPoolOwnsShares"] / lens_contract_df["actualLPTotalSupply"]
    )
    lens_contract_df["reserve_amounts"] = lens_contract_df["vaultAddress"].apply(
        lambda x: acutal_reserves_state[(x, "underlyingReserves_amounts")]
    )
    lens_contract_df["reserve_tokens"] = lens_contract_df["vaultAddress"].apply(
        lambda x: acutal_reserves_state[(x, "underlyingReserves_tokens")]
    )

    raw_base_token_value_by_destination = []

    def _extract_proportional_ownership_of_reserve_tokens_in_underlying_pools(row: pd.Series):
        for reserve_amount, reserve_token_address in zip(row["reserve_amounts"], row["reserve_tokens"]):
            raw_base_token_value_by_destination.append(
                {
                    "token_address": Web3.toChecksumAddress(reserve_token_address),
                    "reserve_amount": int(int(reserve_amount) * row["portion_ownership"]),
                    "vault_address": row["vaultAddress"],
                    "autopool_symbol": row["autopool_symbol"],
                }
            )

    lens_contract_df.apply(
        lambda row: _extract_proportional_ownership_of_reserve_tokens_in_underlying_pools(row), axis=1
    )
    reserve_token_ownership_df = pd.DataFrame.from_records(raw_base_token_value_by_destination)

    token_df = get_full_table_as_df(Tokens, where_clause=Tokens.chain_id == chain.chain_id)

    reserve_token_ownership_df = reserve_token_ownership_df[reserve_token_ownership_df["reserve_amount"] > 0]
    raw_balance_by_token = reserve_token_ownership_df.groupby("token_address")["reserve_amount"].sum()
    return lens_contract_df, reserve_token_ownership_df, token_df, raw_balance_by_token

,token_address,reserve_amount,vault_address,autopool_symbol
0,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,477668499845074452480,0xc4eb861e7b66f593482a3d7e8adc314f6eeda30b,autoETH
1,0x88794C65550DeB6b4087B7552eCf295113794410,720385906514451783620624660824064,0xc4eb861e7b66f593482a3d7e8adc314f6eeda30b,autoETH
2,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,257108860291401908224,0xc4eb861e7b66f593482a3d7e8adc314f6eeda30b,autoETH
3,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,581617675762953420800,0x1ea622fa030e4a78f4cc2f305dd3c08da3f08573,autoETH
4,0xA35b1B31Ce002FBF2058D22F30f95D405200A15b,984277202008518426624,0x1ea622fa030e4a78f4cc2f305dd3c08da3f08573,autoETH
8,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,397810602273961607168,0xba1462f43c6f60ebd1c62735c94e428ad073e01a,autoETH
9,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,657600640950121398272,0xba1462f43c6f60ebd1c62735c94e428ad073e01a,autoETH
10,0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6,1270999094834193235968,0xe4433d00cf48bfe0c672d9949f2cd2c008bffc04,autoETH
11,0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84,748983918883327180800,0xe4433d00cf48bfe0c672d9949f2cd2c008bffc04,autoETH
12,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,1087485510103234248704,0x5c6aeb9ef0d5bba4e6691f381003503fd0d45126,autoETH


In [7]:
token_df

,token_address,chain_id,symbol,name,decimals
0,0x38De22a3175708D45E7c7c64CD78479C8B56f76E,1,DOLA/USR,DOLA/USR,18
1,0xB91159aa527D4769CB9FAf3e4ADB760c7E8C8Ea7,1,ETHx/wstETH,Balancer ETHx/wstETH,18
2,0x853d955aCEf822Db058eb8505911ED77F175b99e,1,FRAX,Frax,18
3,0xff17dAb22F1E61078aBa2623c89cE6110E878B3c,1,dola-save,DOLA+scrvUSD,18
4,0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD,1,sUSDS,Savings USDS,18
...,...,...,...,...,...
83,0x9EfE1A1Cbd6Ca51Ee8319AFc4573d253C3B732af,1,wETHrETH,wETH/rETH,18
84,0x167478921b907422F8E88B43C4Af2B8BEa278d3A,1,MtEthena,sDAI/sUSDe,18
85,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,1,pxethweth,pxETH/wETH,18
86,0x856c4Efb76C1D1AE02e20CEB03A2A6a08b0b8dC3,1,OETH,Origin Ether,18


token_address
0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6                2701534747213665714176
0x05ff47AFADa98a98982113758878F9A8B9FddA0a     629097718663375158586159212265472
0x0655977FEb2f289A4aB78af67BAB0d17aAb84367             3041559843208172390055936
0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f             4447744051768962161049600
0x57aB1E0003F623289CD798B1824Be09a793e4Bec             1105629417473955669737472
0x596192bB6e41802428Ac943D2f1476C1Af25CC0E     110350233524654222785049454968832
0x865377367054516e17014CcdED1e7d814EDC9ce4              123671167088402112708608
0x88794C65550DeB6b4087B7552eCf295113794410    1070666544192624676900245413560320
0x9D39A5DE30e57443BfF2A8307A4256c8797A3497              353871701284104815247360
0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78                  20159626845610848256
0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48                         7437714742941
0xA35b1B31Ce002FBF2058D22F30f95D405200A15b                 984277202008518426624
0xBC6DA0FE9aD5

In [ ]:
row = df.iloc[-1]


raw_base_token_value_by_destination

[{'token_address': '0x6Bb7a212910682DCFdbd5BCBb3e28FB4E8da10Ee',
  'reserve_amount': 4320577737669641439805440,
  'vault_address': '0x1efc63fa8718cac1df88273b08616cf1e9ff03dd'},
 {'token_address': '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913',
  'reserve_amount': 2314278698162,
  'vault_address': '0x1efc63fa8718cac1df88273b08616cf1e9ff03dd'}]

In [ ]:
# def row_to_token_ownership(row:pd.Series) -> None:


# row

vaultAddress                           0x1efc63fa8718cac1df88273b08616cf1e9ff03dd
autoPoolOwnsShares                                      6559803521499856885190855
actualLPTotalSupply                                    34159762272739833940659751
autopool_base_asset_decimals                                                    6
autopool_symbol                                                           baseUSD
portion_ownership                                                        0.192033
reserve_amounts                      (22499135517696352608063688, 12051460063244)
reserve_tokens                  (0x6bb7a212910682dcfdbd5bcbb3e28fb4e8da10ee, 0...
Name: 18, dtype: object

In [3]:
# you need to run your whole loop again

In [ ]:
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_token_values_table import (
    _build_safe_price_calls,
)
from mainnet_launch.database.schema.full import *

In [5]:
def _id_with_bool_success(success, x):
    if success:
        return x


# struct StakingIncentiveStats {
#     // time-weighted average total supply to prevent spikes/attacks from impacting rebalancing
#     uint256 safeTotalSupply;
#     // rewardTokens, annualizedRewardAmounts, and periodFinishForRewards will match indexes
#     // they are split to workaround an issue with forge having nested structs
#     // address of the reward tokens
#     address[] rewardTokens;
#     // the annualized reward rate for the reward token
#     uint256[] annualizedRewardAmounts;
#     // the timestamp for when the rewards are set to terminate
#     uint40[] periodFinishForRewards;
#     // incentive rewards score. max 48, min 0
#     uint8 incentiveCredits;
# }

# struct DexLSTStatsData {
#     uint256 lastSnapshotTimestamp;
#     uint256 feeApr;
#     uint256[] reservesInEth;
#     StakingIncentiveStats stakingIncentiveStats;
#     ILSTStats.LSTStatsData[] lstStatsData;
# }


def build_proxy_get_stats_calls(chain: ChainData, destination_vault_address: str):
    # (IDexLSTStats.DexLSTStatsData memory queriedStat
    return Call(
        LENS_CONTRACT(chain)[
            f"proxyGetStats(address)((address,uint256,uint256,uint256,uint256,int256,int256,int256,uint256,int256,uint256))",
            destination_vault_address,
        ],
        [(destination_vault_address, _id_with_bool_success)],
    )


case

NameError: name 'case' is not defined

In [ ]:
dests = data["destinations"]

#  'autoPoolOwnsShares': 271788422417713608486,
#  'actualLPTotalSupply': 293866231429156627032,


print(f"{lpTokenName=}")
print(f"{symbols=}")
print(f"{autoPoolOwnsShares=}")
print(f"{actualLPTotalSupply=}")
print(f"{underlyingTokenValueHeld=}")
print(f"{reservesInEth=}")

lpTokenName='Balancer rETH-Aave wETH'
symbols=[{'symbol': 'rETH'}, {'symbol': 'WETH'}]
autoPoolOwnsShares=243982146156687172772
actualLPTotalSupply=513002571068356845744
underlyingTokenValueHeld=[{'valueHeldInEth': 131222789617967283417}, {'valueHeldInEth': 113942579229296713637}]
reservesInEth=(275912108804663976981, 239578333986989015663)


In [ ]:
dests[0][-1]

{'vaultAddress': '0xc3337192143eb496eed206c8c4249f7cdee19078',
 'exchangeName': 'balancerV3',
 'totalSupply': 243982146156687172772,
 'lastSnapshotTimestamp': 1751472185,
 'feeApr': 23096435593060712,
 'lastDebtReportTime': 1751443507,
 'minDebtValue': 245149865595496825804,
 'maxDebtValue': 245160090779875522026,
 'debtValueHeldByVault': 245154978187686173915,
 'queuedForRemoval': False,
 'statsIncomplete': False,
 'isShutdown': False,
 'shutdownStatus': 0,
 'autoPoolOwnsShares': 243982146156687172772,
 'actualLPTotalSupply': 513002571068356845744,
 'dexPool': '0xb7b8b3afc010169779c5c2385ec0eb0477fe3347',
 'lpTokenAddress': '0xb7b8b3afc010169779c5c2385ec0eb0477fe3347',
 'lpTokenSymbol': 'Balancer rETH-Aave wETH',
 'lpTokenName': 'Balancer rETH-Aave wETH',
 'statsSafeLPTotalSupply': 509989208214997104380,
 'statsIncentiveCredits': 168,
 'compositeReturn': 62145722113027740,
 'rewardsTokens': [{'tokenAddress': '0x4158734d47fc9692176b5085e0f52ee0da5d47f1'},
  {'tokenAddress': '0x1509706a

In [ ]:
243982146156687172772 / 513002571068356845744

0.47559634184401955

In [ ]:
int(239578333986989015663 * 0.47559634184401955)

113942579229296721920

In [ ]:
113942579229296713637 - 113942579229296721920

-8283

In [ ]:
dests[0][-1]

{'vaultAddress': '0xc3337192143eb496eed206c8c4249f7cdee19078',
 'exchangeName': 'balancerV3',
 'totalSupply': 243982146156687172772,
 'lastSnapshotTimestamp': 1751472185,
 'feeApr': 23096435593060712,
 'lastDebtReportTime': 1751443507,
 'minDebtValue': 245149865595496825804,
 'maxDebtValue': 245160090779875522026,
 'debtValueHeldByVault': 245154978187686173915,
 'queuedForRemoval': False,
 'statsIncomplete': False,
 'isShutdown': False,
 'shutdownStatus': 0,
 'autoPoolOwnsShares': 243982146156687172772,
 'actualLPTotalSupply': 513002571068356845744,
 'dexPool': '0xb7b8b3afc010169779c5c2385ec0eb0477fe3347',
 'lpTokenAddress': '0xb7b8b3afc010169779c5c2385ec0eb0477fe3347',
 'lpTokenSymbol': 'Balancer rETH-Aave wETH',
 'lpTokenName': 'Balancer rETH-Aave wETH',
 'statsSafeLPTotalSupply': 509989208214997104380,
 'statsIncentiveCredits': 168,
 'compositeReturn': 62145722113027740,
 'rewardsTokens': [{'tokenAddress': '0x4158734d47fc9692176b5085e0f52ee0da5d47f1'},
  {'tokenAddress': '0x1509706a